Team 12  
Authors: Lam Kwun Yuk, Sha Yu Hin  
IDs: 20512073, 20516835  

Most code used are from tutorials. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import keras
from keras.utils import to_categorical
import numpy as np
import pickle as pkl
import tensorflow as tf
from nltk.stem import PorterStemmer
from numpy.random import seed
from collections import Counter
from itertools import chain
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Embedding, Dropout, BatchNormalization, Activation, Input, \
    Conv1D, MaxPool1D, Flatten, Concatenate, Add, Average,Bidirectional, SimpleRNN, LSTM, GRU, TimeDistributed
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/4901k-project/val.pkl
/kaggle/input/4901k-project/test.pkl
/kaggle/input/4901k-project/train.pkl


In [2]:
def get_onehot_vector(feats, feats_dict):
    """
    :param feats: a list of features, type: list
    :param feats_dict: a dict from features to indices, type: dict
    return a feature vector,
    """
    # initialize the vector as all zeros
    vector = np.zeros(len(feats_dict), dtype=np.float)
    for f in feats:
        # get the feature index, return -1 if the feature is not existed
        f_idx = feats_dict.get(f, -1)
        if f_idx != -1:
            # set the corresponding element as 1
            vector[f_idx] = 1
    return vector
def stem(tokens):
    """
    :param tokens: a list of tokens, type: list
    return a list of stemmed words, type: list
    e.g.
    Input: ['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
    Output: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
    """
    ### equivalent code
    # results = list()
    # for token in tokens:
    #     results.append(ps.stem(token))
    # return results

    return [ps.stem(token) for token in tokens]
def n_gram(tokens, n=1):
    """
    :param tokens: a list of tokens, type: list
    :param n: the corresponding n-gram, type: int
    return a list of n-gram tokens, type: list
    e.g.
    Input: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.'], 2
    Output: ['text mine', 'mine is', 'is to', 'to identifi', 'identifi use', 'use inform', 'inform .']
    """
    if n == 1:
        return tokens
    else:
        results = list()
        for i in range(len(tokens)-n+1):
            # tokens[i:i+n] will return a sublist from i th to i+n th (i+n th is not included)
            results.append(" ".join(tokens[i:i+n]))
        return results


In [3]:
test_dict = pkl.load(open('/kaggle/input/4901k-project/test.pkl', "rb"))
train_dict = pkl.load(open('/kaggle/input/4901k-project/train.pkl', "rb"))
val_dict = pkl.load(open('/kaggle/input/4901k-project/val.pkl', "rb"))
print("keys in train_dict:", train_dict.keys())
print("keys in val_dict:", val_dict.keys())
print("keys in test_dict:", test_dict.keys())

keys in train_dict: dict_keys(['id', 'word_seq', 'tag_seq'])
keys in val_dict: dict_keys(['id', 'word_seq', 'tag_seq'])
keys in test_dict: dict_keys(['id', 'word_seq'])


In [4]:
# an entry of the dataset
print(train_dict["word_seq"][0])
print(train_dict["tag_seq"][0])
print("index:", train_dict["id"][0])
zipped = zip(train_dict["word_seq"][0], train_dict["tag_seq"][0])
print(*zipped)

['Protection', 'of', 'calves', 'against', 'fatal', 'enteric', 'colibacillosis', 'by', 'orally', 'administered', 'Escherichia', 'coli', 'K99', '-', 'specific', 'monoclonal', 'antibody', '.', 'A', 'monoclonal', 'antibody', '(', 'MCA', ')', 'to', 'enterotoxigenic', 'Escherichia', 'coli', 'K99', 'antigen', 'agglutinated', 'K99+', 'enterotoxigenic', 'E', '.', 'coli', 'strains', 'B44', '(', 'O9', ':', 'K30', ';', 'K99', ';', 'F41', ':', 'H-', ')', 'and', 'B41', '(', 'O101', ':', 'K99', ';', 'F41', ':', 'H-', ')', 'grown', 'at', '37', 'degrees', 'C', 'but', 'not', 'at', '18', 'degrees', 'C.', 'The', 'MCA', ',', 'which', 'was', 'characterized', 'as', 'immunoglobulin', 'G1', ',', 'reacted', 'specifically', 'with', 'K99', 'antigen', 'in', 'an', 'enzyme-linked', 'immunosorbent', 'assay', 'and', 'precipitated', 'radiolabeled', 'K99', 'antigen', '.', 'A', 'total', 'of', '45', 'colostrum', '-fed', 'and', 'colostrum', '-deprived', 'calves', 'were', 'used', 'in', 'three', 'separate', 'trials', 'to', '

In [5]:
# all the NER tags:
from itertools import chain
print("count of the NER tags:", len(set(chain(*train_dict["tag_seq"]))))
print("all the NER tags:", set(chain(*train_dict["tag_seq"])))

count of the NER tags: 65
all the NER tags: {'PERSON', 'GROUP', 'EUKARYOTE', 'RESEARCH_ACTIVITY', 'IMMUNE_RESPONSE', 'MOLECULAR_FUNCTION', 'EXPERIMENTAL_MODEL_OF_DISEASE', 'EDUCATIONAL_ACTIVITY', 'VIRAL_PROTEIN', 'HUMAN-CAUSED_PHENOMENON_OR_PROCESS', 'FAC', 'PRODUCT', 'WILDLIFE', 'LIVESTOCK', 'GOVERNMENTAL_OR_REGULATORY_ACTIVITY', 'ORG', 'LABORATORY_OR_TEST_RESULT', 'TISSUE', 'GPE', 'ANATOMICAL_STRUCTURE', 'CARDINAL', 'MONEY', 'EVENT', 'DATE', 'BACTERIUM', 'ORGANISM', 'LOC', 'LAW', 'THERAPEUTIC_OR_PREVENTIVE_PROCEDURE', 'BODY_SUBSTANCE', 'ORGAN_OR_TISSUE_FUNCTION', 'QUANTITY', 'TIME', 'GENE_OR_GENOME', 'BODY_PART_ORGAN_OR_ORGAN_COMPONENT', 'NORP', 'LANGUAGE', 'GROUP_ATTRIBUTE', 'O', 'PHYSICAL_SCIENCE', 'CELL_OR_MOLECULAR_DYSFUNCTION', 'CELL_FUNCTION', 'CHEMICAL', 'MACHINE_ACTIVITY', 'PERCENT', 'SOCIAL_BEHAVIOR', 'DISEASE_OR_SYNDROME', 'CORONAVIRUS', 'SUBSTRATE', 'MATERIAL', 'INJURY_OR_POISONING', 'DAILY_OR_RECREATIONAL_ACTIVITY', 'INDIVIDUAL_BEHAVIOR', 'WORK_OF_ART', 'EVOLUTION', 'CELL

In [6]:
train_dict['word_seq'] = [stem(tokens) for tokens in train_dict["word_seq"]]
val_dict["word_seq"] = [stem(tokens) for tokens in val_dict["word_seq"]]
test_dict["word_seq"] = [stem(tokens) for tokens in test_dict["word_seq"]]

In [7]:
vocab_dict = {'_unk_': 0, '_w_pad_': 1}

for doc in train_dict['word_seq']:
    for word in doc:
        if(word not in vocab_dict):
            vocab_dict[word] = len(vocab_dict)

tag_dict = {'_t_pad_': 0} # add a padding token

for tag_seq in train_dict['tag_seq']:
    for tag in tag_seq:
        if(tag not in tag_dict):
            tag_dict[tag] = len(tag_dict)
word2idx = vocab_dict
idx2word = {v:k for k,v in word2idx.items()}
tag2idx = tag_dict
idx2tag = {v:k for k,v in tag2idx.items()}            

print("size of word vocab:", len(vocab_dict), "size of tag_dict:", len(tag_dict))

size of word vocab: 55469 size of tag_dict: 65


In [8]:
# The maximum length of a sentence is set to 128
max_sent_length = 128

train_tokens = np.array([[word2idx[w] for w in doc] for doc in train_dict['word_seq']])
val_tokens = np.array([[word2idx.get(w, 0) for w in doc] for doc in val_dict['word_seq']])
test_tokens = np.array([[word2idx.get(w, 0) for w in doc] for doc in test_dict['word_seq']])


train_tags = [[tag2idx[t] for t in t_seq] for t_seq in train_dict['tag_seq']]
train_tags = np.array([to_categorical(t_seq, num_classes=len(tag_dict)) for t_seq in train_tags])

val_tags = [[tag2idx[t] for t in t_seq] for t_seq in val_dict['tag_seq']]
val_tags = np.array([to_categorical(t_seq, num_classes=len(tag_dict)) for t_seq in val_tags])

In [9]:
print("training size:", train_tokens.shape, "tag size:", train_tags.shape)
print("validating size:", val_tokens.shape, "tag size:", val_tags.shape)

training size: (23600, 128) tag size: (23600, 128, 65)
validating size: (2950, 128) tag size: (2950, 128, 65)


In [10]:
def build_CNN(input_length, vocab_size, embedding_size,
              hidden_size, output_size,
              kernel_sizes, num_filters, num_mlp_layers,
              padding="valid",
              strides=1,
              activation="relu",
              dropout_rate=0.0,
              batch_norm=False,
              l2_reg=0.0,
              loss="categorical_crossentropy",
              optimizer="SGD",
              learning_rate=0.1,
              metric="accuracy"):
    """
    :param input_length: the maximum length of sentences, type: int
    :param vocab_size: the vacabulary size, type: int
    :param embedding_size: the dimension of word representations, type: int
    :param hidden_size: the dimension of the hidden states, type: int
    :param output_size: the dimension of the prediction, type: int
    :param kernel_sizes: the kernel sizes of convolutional layers, type: list
    :param num_filters: the number of filters for each kernel, type: int
    :param num_mlp_layers: the number of layers of the MLP, type: int
    :param padding: the padding method in convolutional layers, type: str
    :param strides: the strides in convolutional layers, type: int
    :param activation: the activation type, type: str
    :param dropout_rate: the probability of dropout, type: float
    :param batch_norm: whether to enable batch normalization, type: bool
    :param l2_reg: the weight for the L2 regularizer, type: str
    :param loss: the training loss, type: str
    :param optimizer: the optimizer, type: str
    :param learning_rate: the learning rate for the optimizer, type: float
    :param metric: the metric, type: str
    return a CNN for text classification,
    # activation document: https://keras.io/activations/
    # dropout document: https://keras.io/layers/core/#dropout
    # embedding document: https://keras.io/layers/embeddings/#embedding
    # convolutional layers document: https://keras.io/layers/convolutional
    # pooling layers document: https://keras.io/layers/pooling/
    # batch normalization document: https://keras.io/layers/normalization/
    # losses document: https://keras.io/losses/
    # optimizers document: https://keras.io/optimizers/
    # metrics document: https://keras.io/metrics/
    """
    x = Input(shape=(input_length,))
    print(input_length,vocab_size,embedding_size,output_size)
    ################################
    ###### Word Representation #####
    ################################
    # word representation layer
    emb = Embedding(input_dim=vocab_size,
                    output_dim=embedding_size,
                    input_length=input_length,
                    embeddings_initializer=keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=0))(x)
    
    ################################
    ########### Conv-Pool ##########
    ################################
    # convolutional and pooling layers
    cnn_results = list()
    for kernel_size in kernel_sizes:
        # add convolutional layer
        conv = Conv1D(filters=num_filters,
                      kernel_size=(kernel_size,),
                      padding=padding,
                      strides=strides)(emb)
        # add batch normalization layer
        if batch_norm:
            conv = BatchNormalization()(conv)
        # add activation
        conv = Activation(activation)(conv)
        # add max-pooling
        #maxpool = MaxPool1D(pool_size=(input_length-kernel_size)//strides+1)(conv)
        #cnn_results.append(Flatten()(maxpool))
        cnn_results.append(conv)
    
    ################################
    ##### Fully Connected Layer ####
    ################################
    h = Average()(cnn_results) if len(kernel_sizes) > 1 else cnn_results[0]
    #h = Concatenate()(cnn_results) if len(kernel_sizes) > 1 else cnn_results[0]
    h = Dropout(dropout_rate, seed=0)(h)
    #h = Embedding(input_dim=vocab_size,
    #                output_dim=embedding_size,
    #                input_length=input_length,
    #                embeddings_initializer=keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=0))(h)
    # multi-layer perceptron
    for i in range(num_mlp_layers-1):
        new_h = Dense(hidden_size,
                      kernel_regularizer=keras.regularizers.l2(l2_reg))(h)
        # add batch normalization layer
        if batch_norm:
            new_h = BatchNormalization()(new_h)
        # add skip connection
        if i == 0:
            h = new_h
        else:
            h = Add()([h, new_h])
        # add activation
        h = Activation(activation)(h)
    y = Dense(output_size,
              activation="softmax")(h)
    
    # set the loss, the optimizer, and the metric
    if optimizer == "SGD":
        optimizer = keras.optimizers.SGD(lr=learning_rate)
    elif optimizer == "RMSprop":
        optmizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer == "Adam":
        optmizer = keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        raise NotImplementedError
    model = Model(x, y)
    model.compile(loss=loss, optimizer=optimizer, metrics=[metric])
    
    return model

In [11]:
def build_RNN(input_length, vocab_size, embedding_size,
              hidden_size, output_size,
              num_rnn_layers, num_mlp_layers,
              rnn_type="lstm",
              bidirectional=False,
              activation="tanh",
              dropout_rate=0.0,
              batch_norm=False,
              l2_reg=0.0,
              loss="categorical_crossentropy",
              optimizer="Adam",
              learning_rate=0.001,
              metric="accuracy",
              return_sequences=True):
    """
    :param input_length: the maximum length of sentences, type: int
    :param vocab_size: the vacabulary size, type: int
    :param embedding_size: the dimension of word representations, type: int
    :param hidden_size: the dimension of the hidden states, type: int
    :param output_size: the dimension of the prediction, type: int
    :param num_rnn_layers: the number of layers of the RNN, type: int
    :param num_mlp_layers: the number of layers of the MLP, type: int
    :param rnn_type: the type of RNN, type: str
    :param bidirectional: whether to use bidirectional rnn, type: bool
    :param activation: the activation type, type: str
    :param dropout_rate: the probability of dropout, type: float
    :param batch_norm: whether to enable batch normalization, type: bool
    :param l2_reg: the weight for the L2 regularizer, type: str
    :param loss: the training loss, type: str
    :param optimizer: the optimizer, type: str
    :param learning_rate: the learning rate for the optimizer, type: float
    :param metric: the metric, type: str
    return a RNN for text classification,
    # activation document: https://keras.io/activations/
    # dropout document: https://keras.io/layers/core/#dropout
    # embedding document: https://keras.io/layers/embeddings/#embedding
    # recurrent layers document: https://keras.io/layers/recurrent
    # batch normalization document: https://keras.io/layers/normalization/
    # losses document: https://keras.io/losses/
    # optimizers document: https://keras.io/optimizers/
    # metrics document: https://keras.io/metrics/
    """
    x = Input(shape=(input_length,))
    
    ################################
    ###### Word Representation #####
    ################################
    # word representation layer
    emb = Embedding(input_dim=vocab_size,
                    output_dim=embedding_size,
                    input_length=input_length,
                    embeddings_initializer=keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=0))(x)
    
    ################################
    ####### Recurrent Layers #######
    ################################
    # recurrent layers
    # Referennce: https://keras.io/api/layers/#recurrent-layers
    if rnn_type == "rnn":
        fn = SimpleRNN
    elif rnn_type == "lstm":
        fn = LSTM
    elif rnn_type == "gru":
        fn = GRU
    else:
        raise NotImplementedError
        
    h = emb
    for i in range(num_rnn_layers):
        is_last = (i == num_rnn_layers-1)
        if bidirectional:
            h = Bidirectional(fn(hidden_size,
                   kernel_initializer=keras.initializers.glorot_uniform(seed=0),
                   recurrent_initializer=keras.initializers.Orthogonal(gain=1.0, seed=0),
                   return_sequences=True))(h)
            # return_sequences:
            # Boolean. Whether to return the last output. in the output sequence, or the full sequence.
            # [h_1, h_2, ..., h_n] or h_n
        else:
            h = fn(hidden_size,
                   kernel_initializer=keras.initializers.glorot_uniform(seed=0),
                   recurrent_initializer=keras.initializers.Orthogonal(gain=1.0, seed=0),
                   return_sequences=True)(h)
        h = Dropout(dropout_rate, seed=0)(h)

    ################################
    #### Fully Connected Layers ####
    ################################
    # multi-layer perceptron
    for i in range(num_mlp_layers-1):
        new_h = Dense(hidden_size,
                      kernel_initializer=keras.initializers.he_normal(seed=0),
                      bias_initializer="zeros",
                      kernel_regularizer=keras.regularizers.l2(l2_reg))(h)
        # add batch normalization layer
        if batch_norm:
            new_h = BatchNormalization()(new_h)
        # add residual connection
        if i == 0:
            h = new_h
        else:
            h = Add()([h, new_h])
        # add activation
        h = Activation(activation)(h)
    y = Dense(output_size,
              activation="softmax",
              kernel_initializer=keras.initializers.he_normal(seed=0),
              bias_initializer="zeros")(h)
    
    # set the loss, the optimizer, and the metric
    if optimizer == "SGD":
        optimizer = keras.optimizers.SGD(lr=learning_rate)
    elif optimizer == "RMSprop":
        optmizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer == "Adam":
        optmizer = keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        raise NotImplementedError
    model = Model(x, y)
    model.compile(loss=loss, optimizer=optimizer, metrics=[metric])
    
    return model

In [57]:
# Provided function to test accuracy
# You could check the validation accuracy to select the best of your models
def calc_accuracy(preds, tags, padding_id="_t_pad_"):
    """
        Input:
            preds (np.narray): (num_data, length_sentence)
            tags  (np.narray): (num_data, length_sentence)
        Output:
            Proportion of correct prediction. The padding tokens are filtered out.
    """
    preds_flatten = preds.flatten()
    tags_flatten = tags.flatten()
    non_padding_idx = np.where(tags_flatten!=padding_id)[0]
    
    return sum(preds_flatten[non_padding_idx]==tags_flatten[non_padding_idx])/len(non_padding_idx)

def evaluate(pred_file, ground_file):
    file_dict = pkl.load(open(ground_file, "rb"))
    file_preds = pd.read_csv(pred_file)
    return calc_accuracy(np.array([json.loads(line) for line in file_preds["labels"]]), 
              np.array(file_dict["tag_seq"]))

# CNN1

In [13]:
os.makedirs("models", exist_ok=True)

seed(0)
tf.random.set_seed(0)
print(seed)

model = build_CNN(input_length=max_sent_length, vocab_size=len(vocab_dict),
                  embedding_size=128, hidden_size=128, output_size=len(tag_dict),
                  kernel_sizes=[1], num_filters=128, num_mlp_layers=2,
                  activation="relu",optimizer="Adam",learning_rate=0.05)
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join("models", "weights_cnn1.hdf5"),
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True)

print(model.summary())
cnn_history = model.fit(train_tokens, train_tags,
                    validation_split=0.1,
                    epochs=10, batch_size=100, verbose=1,
                    callbacks=[checkpointer])
model = keras.models.load_model(os.path.join("models", "weights_cnn1.hdf5"))

train_score = model.evaluate(train_tokens, train_tags,
                             batch_size=100)
test_score = model.evaluate(val_tokens, val_tags,
                            batch_size=100)
print("training loss:", train_score[0], "training accuracy", train_score[1])
print("test loss:", test_score[0], "test accuracy", test_score[1])

<built-in method seed of numpy.random.mtrand.RandomState object at 0x7fc0094ea9e0>
128 55469 128 65
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 128, 128)          7100032   
_________________________________________________________________
conv1d (Conv1D)              (None, 128, 128)          16512     
_________________________________________________________________
activation (Activation)      (None, 128, 128)          0         
_________________________________________________________________
dropout (Dropout)            (None, 128, 128)          0         
_________________________________________________________________
dense (Dense)                (None, 128, 128)          16512     
____________________

In [14]:
train_matrix = model.predict(val_tokens,batch_size=100, verbose=1, callbacks=[checkpointer])
train_tags_by_idx = np.argmax(train_matrix, axis=2)
train_labels = np.array([[idx2tag[p] for p in preds] for preds in train_tags_by_idx])
print(train_labels)

print("CNN. Acc:", 
      calc_accuracy(train_labels, 
                    np.array([val_dict['tag_seq']])))

30/30 [==============================] - 0s 6ms/step
[['IMMUNE_RESPONSE' 'O' 'O' ... 'O' 'CHEMICAL' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ...
 ['O' 'DATE' 'CORONAVIRUS' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ['GENE_OR_GENOME' 'O' 'O' ... 'O' 'O' 'O']]
CNN. Acc: 0.8400792617663224


# RNN1

In [15]:
embedding_size = 128
hidden_size = 64 
num_rnn_layers = 1
num_mlp_layers = 1
os.makedirs("RNN_models", exist_ok=True)
model = build_RNN(max_sent_length, len(vocab_dict), embedding_size,
              hidden_size, len(tag_dict),
              num_rnn_layers, num_mlp_layers,
              rnn_type="lstm",
              bidirectional=False,
              activation="tanh",
              dropout_rate=0.0,
              batch_norm=False,
              l2_reg=0.0,
              loss="categorical_crossentropy",
              optimizer="Adam",
              learning_rate=0.001,
              metric="accuracy")
print(model.summary())


Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 128, 128)          7100032   
_________________________________________________________________
lstm (LSTM)                  (None, 128, 64)           49408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 128, 65)           4225      
Total params: 7,153,665
Trainable params: 7,153,665
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join("models", "weights_RNN1.hdf5"),
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True)
earlystopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1)

np.random.seed(0)
tf.random.set_seed(0)
rnn_history = model.fit(train_tokens, train_tags,
                    validation_split=0.1,
                    epochs=20, batch_size=100, verbose=1,
                    callbacks=[checkpointer, earlystopping])
model = keras.models.load_model(os.path.join("models", "weights_RNN1.hdf5"))

train_score = model.evaluate(train_tokens, train_tags,
                             batch_size=100)
test_score = model.evaluate(val_tokens, val_tags,
                            batch_size=100)
print("training loss:", train_score[0], "training accuracy", train_score[1])
print("test loss:", test_score[0], "test accuracy", test_score[1])

Epoch 1/20
213/213 [==============================] - ETA: 0s - loss: 1.3318 - accuracy: 0.7488
Epoch 00001: val_accuracy improved from -inf to 0.78878, saving model to models/weights_RNN1.hdf5
213/213 [==============================] - 21s 99ms/step - loss: 1.3318 - accuracy: 0.7488 - val_loss: 0.8477 - val_accuracy: 0.7888
Epoch 2/20
213/213 [==============================] - ETA: 0s - loss: 0.7337 - accuracy: 0.8189
Epoch 00002: val_accuracy improved from 0.78878 to 0.83774, saving model to models/weights_RNN1.hdf5
213/213 [==============================] - 20s 92ms/step - loss: 0.7337 - accuracy: 0.8189 - val_loss: 0.6404 - val_accuracy: 0.8377
Epoch 3/20
213/213 [==============================] - ETA: 0s - loss: 0.5789 - accuracy: 0.8514
Epoch 00003: val_accuracy improved from 0.83774 to 0.85614, saving model to models/weights_RNN1.hdf5
213/213 [==============================] - 20s 96ms/step - loss: 0.5789 - accuracy: 0.8514 - val_loss: 0.5479 - val_accuracy: 0.8561
Epoch 4/20
21

In [17]:
train_matrix = model.predict(val_tokens,batch_size=100, verbose=1, callbacks=[checkpointer])
train_tags_by_idx = np.argmax(train_matrix, axis=2)
train_labels = np.array([[idx2tag[p] for p in preds] for preds in train_tags_by_idx])
print(train_labels)
print("RNN:LSTM. Acc:", calc_accuracy(train_labels, np.array([val_dict['tag_seq']])))

30/30 [==============================] - 0s 8ms/step
[['IMMUNE_RESPONSE' 'O' 'O' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ...
 ['O' 'DATE' 'CORONAVIRUS' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ['GENE_OR_GENOME' 'O' 'O' ... 'O' 'O' 'O']]
RNN:LSTM. Acc: 0.8633602360496399


# BILSTM1

In [18]:
embedding_size = 128
hidden_size = 64 
num_rnn_layers = 1
num_mlp_layers = 1
model = build_RNN(max_sent_length, len(vocab_dict), embedding_size,
              hidden_size, len(tag_dict),
              num_rnn_layers, num_mlp_layers,
              rnn_type="lstm",
              bidirectional=True,
              activation="tanh",
              dropout_rate=0.0,
              batch_norm=False,
              l2_reg=0.0,
              loss="categorical_crossentropy",
              optimizer="Adam",
              learning_rate=0.001,
              metric="accuracy")
print(model.summary())


Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 128, 128)          7100032   
_________________________________________________________________
bidirectional (Bidirectional (None, 128, 128)          98816     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
dense_3 (Dense)              (None, 128, 65)           8385      
Total params: 7,207,233
Trainable params: 7,207,233
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join("models", "weights_BILSTM1.hdf5"),
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True)
earlystopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1)

np.random.seed(0)
tf.random.set_seed(0)
rnn_history = model.fit(train_tokens, train_tags,
                    validation_split=0.1,
                    epochs=20, batch_size=100, verbose=1,
                    callbacks=[checkpointer, earlystopping])
model = keras.models.load_model(os.path.join("models", "weights_BILSTM1.hdf5"))

train_score = model.evaluate(train_tokens, train_tags,
                             batch_size=100)
test_score = model.evaluate(val_tokens, val_tags,
                            batch_size=100)
print("training loss:", train_score[0], "training accuracy", train_score[1])
print("test loss:", test_score[0], "test accuracy", test_score[1])

Epoch 1/20
213/213 [==============================] - ETA: 0s - loss: 1.2146 - accuracy: 0.7637
Epoch 00001: val_accuracy improved from -inf to 0.79464, saving model to models/weights_BILSTM1.hdf5
213/213 [==============================] - 24s 113ms/step - loss: 1.2146 - accuracy: 0.7637 - val_loss: 0.8219 - val_accuracy: 0.7946
Epoch 2/20
213/213 [==============================] - ETA: 0s - loss: 0.7147 - accuracy: 0.8213
Epoch 00002: val_accuracy improved from 0.79464 to 0.84164, saving model to models/weights_BILSTM1.hdf5
213/213 [==============================] - 22s 103ms/step - loss: 0.7147 - accuracy: 0.8213 - val_loss: 0.6120 - val_accuracy: 0.8416
Epoch 3/20
213/213 [==============================] - ETA: 0s - loss: 0.5423 - accuracy: 0.8584
Epoch 00003: val_accuracy improved from 0.84164 to 0.86372, saving model to models/weights_BILSTM1.hdf5
213/213 [==============================] - 22s 103ms/step - loss: 0.5423 - accuracy: 0.8584 - val_loss: 0.5081 - val_accuracy: 0.8637
E

In [20]:
train_matrix = model.predict(val_tokens,batch_size=100, verbose=1, callbacks=[checkpointer])
train_tags_by_idx = np.argmax(train_matrix, axis=2)
train_labels = np.array([[idx2tag[p] for p in preds] for preds in train_tags_by_idx])
print(train_labels)
print("BI-LSTM1. Acc:", 
      calc_accuracy(train_labels, 
                    np.array([val_dict['tag_seq']])))

30/30 [==============================] - 0s 10ms/step
[['IMMUNE_RESPONSE' 'O' 'O' ... 'O' 'ORGANISM' 'O']
 ['GENE_OR_GENOME' 'O' 'O' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ...
 ['O' 'DATE' 'CORONAVIRUS' ... 'O' 'O' 'O']
 ['DATE' 'O' 'O' ... 'O' 'O' 'O']
 ['CHEMICAL' 'O' 'O' ... 'O' 'O' 'O']]
BI-LSTM1. Acc: 0.8827620122074691


# BILSTM2

In [21]:
embedding_size = 200
hidden_size = 256
num_rnn_layers = 4
num_mlp_layers = 2
model = build_RNN(max_sent_length, len(vocab_dict), embedding_size,
              hidden_size, len(tag_dict),
              num_rnn_layers, num_mlp_layers,
              rnn_type="lstm",
              bidirectional=True,
              activation="tanh",
              dropout_rate=0.0,
              batch_norm=False,
              l2_reg=0.0,
              loss="categorical_crossentropy",
              optimizer="Adam",
              learning_rate=0.001,
              metric="accuracy")
print(model.summary())

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 128, 200)          11093800  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128, 512)          935936    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 512)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128, 512)          1574912   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128, 512)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128, 512)         

In [22]:
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join("models", "weights_BILSTM2.hdf5"),
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True)
earlystopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1)

np.random.seed(0)
tf.random.set_seed(0)
rnn_history = model.fit(train_tokens, train_tags,
                    validation_split=0.1,
                    epochs=20, batch_size=100, verbose=1,
                    callbacks=[checkpointer, earlystopping])
model = keras.models.load_model(os.path.join("models", "weights_BILSTM2.hdf5"))

train_score = model.evaluate(train_tokens, train_tags,
                             batch_size=100)
test_score = model.evaluate(val_tokens, val_tags,
                            batch_size=100)
print("training loss:", train_score[0], "training accuracy", train_score[1])
print("test loss:", test_score[0], "test accuracy", test_score[1])

Epoch 1/20
213/213 [==============================] - ETA: 0s - loss: 1.0336 - accuracy: 0.7782
Epoch 00001: val_accuracy improved from -inf to 0.81637, saving model to models/weights_BILSTM2.hdf5
213/213 [==============================] - 70s 328ms/step - loss: 1.0336 - accuracy: 0.7782 - val_loss: 0.6885 - val_accuracy: 0.8164
Epoch 2/20
213/213 [==============================] - ETA: 0s - loss: 0.5729 - accuracy: 0.8467
Epoch 00002: val_accuracy improved from 0.81637 to 0.86621, saving model to models/weights_BILSTM2.hdf5
213/213 [==============================] - 67s 314ms/step - loss: 0.5729 - accuracy: 0.8467 - val_loss: 0.4938 - val_accuracy: 0.8662
Epoch 3/20
213/213 [==============================] - ETA: 0s - loss: 0.4244 - accuracy: 0.8827
Epoch 00003: val_accuracy improved from 0.86621 to 0.88309, saving model to models/weights_BILSTM2.hdf5
213/213 [==============================] - 69s 323ms/step - loss: 0.4244 - accuracy: 0.8827 - val_loss: 0.4259 - val_accuracy: 0.8831
E

In [23]:
train_matrix = model.predict(val_tokens,batch_size=100, verbose=1, callbacks=[checkpointer])
train_tags_by_idx = np.argmax(train_matrix, axis=2)
train_labels = np.array([[idx2tag[p] for p in preds] for preds in train_tags_by_idx])
print(train_labels)
print("BI-LSTM2. Acc:", 
      calc_accuracy(train_labels, 
                    np.array([val_dict['tag_seq']])))

30/30 [==============================] - 2s 70ms/step
[['O' 'O' 'O' ... 'O' 'ORGANISM' 'O']
 ['ORGANISM' 'GENE_OR_GENOME' 'O' ... 'O' 'O' 'O']
 ['O' 'GROUP' 'O' ... 'O' 'O' 'O']
 ...
 ['O' 'DATE' 'CORONAVIRUS' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ['CHEMICAL' 'O' 'O' ... 'O' 'O' 'O']]
BI-LSTM2. Acc: 0.8945124244265092


# BIGRU

In [24]:
embedding_size = 128
hidden_size = 64 
num_rnn_layers = 1
num_mlp_layers = 1
model = build_RNN(max_sent_length, len(vocab_dict), embedding_size,
              hidden_size, len(tag_dict),
              num_rnn_layers, num_mlp_layers,
              rnn_type="gru",
              bidirectional=True,
              activation="tanh",
              dropout_rate=0.0,
              batch_norm=False,
              l2_reg=0.0,
              loss="categorical_crossentropy",
              optimizer="Adam",
              learning_rate=0.001,
              metric="accuracy")
print(model.summary())


Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 128, 128)          7100032   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128, 128)          74496     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
dense_6 (Dense)              (None, 128, 65)           8385      
Total params: 7,182,913
Trainable params: 7,182,913
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join("models", "weights_BIGRU.hdf5"),
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True)
earlystopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1)

np.random.seed(0)
tf.random.set_seed(0)
rnn_history = model.fit(train_tokens, train_tags,
                    validation_split=0.1,
                    epochs=20, batch_size=100, verbose=1,
                    callbacks=[checkpointer, earlystopping])
model = keras.models.load_model(os.path.join("models", "weights_BIGRU.hdf5"))

train_score = model.evaluate(train_tokens, train_tags,
                             batch_size=100)
test_score = model.evaluate(val_tokens, val_tags,
                            batch_size=100)
print("training loss:", train_score[0], "training accuracy", train_score[1])
print("test loss:", test_score[0], "test accuracy", test_score[1])

Epoch 1/20
213/213 [==============================] - ETA: 0s - loss: 1.1677 - accuracy: 0.7747
Epoch 00001: val_accuracy improved from -inf to 0.83097, saving model to models/weights_BIGRU.hdf5
213/213 [==============================] - 22s 104ms/step - loss: 1.1677 - accuracy: 0.7747 - val_loss: 0.6733 - val_accuracy: 0.8310
Epoch 2/20
213/213 [==============================] - ETA: 0s - loss: 0.5732 - accuracy: 0.8502
Epoch 00002: val_accuracy improved from 0.83097 to 0.86226, saving model to models/weights_BIGRU.hdf5
213/213 [==============================] - 21s 100ms/step - loss: 0.5732 - accuracy: 0.8502 - val_loss: 0.5104 - val_accuracy: 0.8623
Epoch 3/20
213/213 [==============================] - ETA: 0s - loss: 0.4537 - accuracy: 0.8741
Epoch 00003: val_accuracy improved from 0.86226 to 0.87485, saving model to models/weights_BIGRU.hdf5
213/213 [==============================] - 19s 91ms/step - loss: 0.4537 - accuracy: 0.8741 - val_loss: 0.4446 - val_accuracy: 0.8749
Epoch 4/

In [26]:
train_matrix = model.predict(val_tokens,batch_size=100, verbose=1, callbacks=[checkpointer])
train_tags_by_idx = np.argmax(train_matrix, axis=2)
train_labels = np.array([[idx2tag[p] for p in preds] for preds in train_tags_by_idx])
print(train_labels)
print("BI-GRU. Acc:", 
      calc_accuracy(train_labels, 
                    np.array([val_dict['tag_seq']])))

30/30 [==============================] - 0s 10ms/step
[['IMMUNE_RESPONSE' 'O' 'O' ... 'O' 'ORGANISM' 'O']
 ['ORGANISM' 'O' 'O' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ...
 ['O' 'DATE' 'CORONAVIRUS' ... 'O' 'O' 'O']
 ['DATE' 'O' 'O' ... 'O' 'O' 'O']
 ['CELL_COMPONENT' 'O' 'O' ... 'O' 'O' 'O']]
BI-GRU. Acc: 0.8907923283867049


# BIGRU2_RELU

In [27]:
embedding_size = 128
hidden_size = 64 
num_rnn_layers = 2
num_mlp_layers = 1
model = build_RNN(max_sent_length, len(vocab_dict), embedding_size,
              hidden_size, len(tag_dict),
              num_rnn_layers, num_mlp_layers,
              rnn_type="gru",
              bidirectional=True,
              activation="relu",
              dropout_rate=0.0,
              batch_norm=False,
              l2_reg=0.0,
              loss="categorical_crossentropy",
              optimizer="Adam",
              learning_rate=0.001,
              metric="accuracy")
print(model.summary())


Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 128, 128)          7100032   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 128, 128)          74496     
_________________________________________________________________
dropout_8 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128, 128)          74496     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
dense_7 (Dense)              (None, 128, 65)         

In [28]:
#Fine tuning for BI-GRU
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join("models", "weights_BIGRU2_RELU.hdf5"),
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True)
earlystopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1)

np.random.seed(0)
tf.random.set_seed(0)
rnn_history = model.fit(train_tokens, train_tags,
                    validation_split=0.1,
                    epochs=20, batch_size=100, verbose=1,
                    callbacks=[checkpointer, earlystopping])
model = keras.models.load_model(os.path.join("models", "weights_BIGRU2_RELU.hdf5"))

train_score = model.evaluate(train_tokens, train_tags,
                             batch_size=100)
test_score = model.evaluate(val_tokens, val_tags,
                            batch_size=100)
print("training loss:", train_score[0], "training accuracy", train_score[1])
print("test loss:", test_score[0], "test accuracy", test_score[1])

Epoch 1/20
213/213 [==============================] - ETA: 0s - loss: 1.0755 - accuracy: 0.7824
Epoch 00001: val_accuracy improved from -inf to 0.83859, saving model to models/weights_BIGRU2_RELU.hdf5
213/213 [==============================] - 26s 120ms/step - loss: 1.0755 - accuracy: 0.7824 - val_loss: 0.6360 - val_accuracy: 0.8386
Epoch 2/20
213/213 [==============================] - ETA: 0s - loss: 0.5329 - accuracy: 0.8602
Epoch 00002: val_accuracy improved from 0.83859 to 0.87074, saving model to models/weights_BIGRU2_RELU.hdf5
213/213 [==============================] - 23s 107ms/step - loss: 0.5329 - accuracy: 0.8602 - val_loss: 0.4760 - val_accuracy: 0.8707
Epoch 3/20
213/213 [==============================] - ETA: 0s - loss: 0.4149 - accuracy: 0.8847
Epoch 00003: val_accuracy improved from 0.87074 to 0.88241, saving model to models/weights_BIGRU2_RELU.hdf5
213/213 [==============================] - 24s 112ms/step - loss: 0.4149 - accuracy: 0.8847 - val_loss: 0.4226 - val_accura

In [29]:
train_matrix = model.predict(val_tokens,batch_size=100, verbose=1, callbacks=[checkpointer])
train_tags_by_idx = np.argmax(train_matrix, axis=2)
train_labels = np.array([[idx2tag[p] for p in preds] for preds in train_tags_by_idx])
print(train_labels)
print("BI-GRU2_RELU. Acc:", 
      calc_accuracy(train_labels, 
                    np.array([val_dict['tag_seq']])))

30/30 [==============================] - 0s 15ms/step
[['IMMUNE_RESPONSE' 'O' 'O' ... 'O' 'ORGANISM' 'O']
 ['ORGANISM' 'ORGANISM' 'O' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ...
 ['O' 'DATE' 'CORONAVIRUS' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'GENE_OR_GENOME' 'O' 'O']
 ['CHEMICAL' 'O' 'O' ... 'O' 'O' 'O']]
BI-GRU2_RELU. Acc: 0.8961555150568429


# BIGRU3_RELU

In [30]:
embedding_size = 128
hidden_size = 64 
num_rnn_layers = 3
num_mlp_layers = 1
model = build_RNN(max_sent_length, len(vocab_dict), embedding_size,
              hidden_size, len(tag_dict),
              num_rnn_layers, num_mlp_layers,
              rnn_type="gru",
              bidirectional=True,
              activation="relu",
              dropout_rate=0.0,
              batch_norm=False,
              l2_reg=0.0,
              loss="categorical_crossentropy",
              optimizer="Adam",
              learning_rate=0.001,
              metric="accuracy")
print(model.summary())


Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 128, 128)          7100032   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 128, 128)          74496     
_________________________________________________________________
dropout_10 (Dropout)         (None, 128, 128)          0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 128, 128)          74496     
_________________________________________________________________
dropout_11 (Dropout)         (None, 128, 128)          0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 128, 128)        

In [31]:
#Fine tuning for BI-GRU
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join("models", "weights_BIGRU3_RELU.hdf5"),
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True)
earlystopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1)

np.random.seed(0)
tf.random.set_seed(0)
rnn_history = model.fit(train_tokens, train_tags,
                    validation_split=0.1,
                    epochs=20, batch_size=100, verbose=1,
                    callbacks=[checkpointer, earlystopping])
model = keras.models.load_model(os.path.join("models", "weights_BIGRU3_RELU.hdf5"))

train_score = model.evaluate(train_tokens, train_tags,
                             batch_size=100)
test_score = model.evaluate(val_tokens, val_tags,
                            batch_size=100)
print("training loss:", train_score[0], "training accuracy", train_score[1])
print("test loss:", test_score[0], "test accuracy", test_score[1])

Epoch 1/20
213/213 [==============================] - ETA: 0s - loss: 1.0467 - accuracy: 0.7840
Epoch 00001: val_accuracy improved from -inf to 0.83728, saving model to models/weights_BIGRU3_RELU.hdf5
213/213 [==============================] - 30s 143ms/step - loss: 1.0467 - accuracy: 0.7840 - val_loss: 0.6435 - val_accuracy: 0.8373
Epoch 2/20
213/213 [==============================] - ETA: 0s - loss: 0.5352 - accuracy: 0.8608
Epoch 00002: val_accuracy improved from 0.83728 to 0.87295, saving model to models/weights_BIGRU3_RELU.hdf5
213/213 [==============================] - 27s 129ms/step - loss: 0.5352 - accuracy: 0.8608 - val_loss: 0.4734 - val_accuracy: 0.8730
Epoch 3/20
213/213 [==============================] - ETA: 0s - loss: 0.4102 - accuracy: 0.8869
Epoch 00003: val_accuracy improved from 0.87295 to 0.88429, saving model to models/weights_BIGRU3_RELU.hdf5
213/213 [==============================] - 28s 132ms/step - loss: 0.4102 - accuracy: 0.8869 - val_loss: 0.4178 - val_accura

In [32]:
train_matrix = model.predict(val_tokens,batch_size=100, verbose=1, callbacks=[checkpointer])
train_tags_by_idx = np.argmax(train_matrix, axis=2)
train_labels = np.array([[idx2tag[p] for p in preds] for preds in train_tags_by_idx])
print(train_labels)
print("BI-GRU3_RELU. Acc:", 
      calc_accuracy(train_labels, 
                    np.array([val_dict['tag_seq']])))

30/30 [==============================] - 1s 20ms/step
[['O' 'O' 'O' ... 'O' 'ORGANISM' 'O']
 ['ORGANISM' 'ORGANISM' 'O' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ...
 ['O' 'DATE' 'CORONAVIRUS' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'CHEMICAL' 'O' 'O']
 ['CELL_COMPONENT' 'O' 'O' ... 'O' 'O' 'O']]
BI-GRU3_RELU. Acc: 0.8956203534959067


# BI-GRU22_RELU

In [33]:
embedding_size = 128
hidden_size = 64 
num_rnn_layers = 2
num_mlp_layers = 2
model = build_RNN(max_sent_length, len(vocab_dict), embedding_size,
              hidden_size, len(tag_dict),
              num_rnn_layers, num_mlp_layers,
              rnn_type="gru",
              bidirectional=True,
              activation="relu",
              dropout_rate=0.0,
              batch_norm=False,
              l2_reg=0.0,
              loss="categorical_crossentropy",
              optimizer="Adam",
              learning_rate=0.001,
              metric="accuracy")
print(model.summary())

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 128, 128)          7100032   
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 128, 128)          74496     
_________________________________________________________________
dropout_13 (Dropout)         (None, 128, 128)          0         
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 128, 128)          74496     
_________________________________________________________________
dropout_14 (Dropout)         (None, 128, 128)          0         
_________________________________________________________________
dense_9 (Dense)              (None, 128, 64)         

In [34]:
#Fine tuning for BI-GRU
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join("models", "weights_BIGRU22_RELU.hdf5"),
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True)
earlystopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1)

np.random.seed(0)
tf.random.set_seed(0)
rnn_history = model.fit(train_tokens, train_tags,
                    validation_split=0.1,
                    epochs=20, batch_size=100, verbose=1,
                    callbacks=[checkpointer, earlystopping])
model = keras.models.load_model(os.path.join("models", "weights_BIGRU22_RELU.hdf5"))

train_score = model.evaluate(train_tokens, train_tags,
                             batch_size=100)
test_score = model.evaluate(val_tokens, val_tags,
                            batch_size=100)
print("training loss:", train_score[0], "training accuracy", train_score[1])
print("test loss:", test_score[0], "test accuracy", test_score[1])

Epoch 1/20
213/213 [==============================] - ETA: 0s - loss: 1.0529 - accuracy: 0.7793
Epoch 00001: val_accuracy improved from -inf to 0.84037, saving model to models/weights_BIGRU22_RELU.hdf5
213/213 [==============================] - 24s 111ms/step - loss: 1.0529 - accuracy: 0.7793 - val_loss: 0.6135 - val_accuracy: 0.8404
Epoch 2/20
213/213 [==============================] - ETA: 0s - loss: 0.5135 - accuracy: 0.8627
Epoch 00002: val_accuracy improved from 0.84037 to 0.87216, saving model to models/weights_BIGRU22_RELU.hdf5
213/213 [==============================] - 22s 103ms/step - loss: 0.5135 - accuracy: 0.8627 - val_loss: 0.4623 - val_accuracy: 0.8722
Epoch 3/20
213/213 [==============================] - ETA: 0s - loss: 0.4019 - accuracy: 0.8862
Epoch 00003: val_accuracy improved from 0.87216 to 0.88311, saving model to models/weights_BIGRU22_RELU.hdf5
213/213 [==============================] - 23s 109ms/step - loss: 0.4019 - accuracy: 0.8862 - val_loss: 0.4149 - val_acc

In [35]:
train_matrix = model.predict(val_tokens,batch_size=100, verbose=1, callbacks=[checkpointer])
train_tags_by_idx = np.argmax(train_matrix, axis=2)
train_labels = np.array([[idx2tag[p] for p in preds] for preds in train_tags_by_idx])
print(train_labels)
print("BI-GRU22_RELU. Acc:", 
      calc_accuracy(train_labels, 
                    np.array([val_dict['tag_seq']])))

30/30 [==============================] - 1s 25ms/step
[['O' 'O' 'O' ... 'O' 'ORGANISM' 'O']
 ['ORGANISM' 'O' 'O' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ...
 ['O' 'DATE' 'CORONAVIRUS' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'CHEMICAL' 'CHEMICAL' 'O']
 ['CHEMICAL' 'O' 'O' ... 'O' 'O' 'O']]
BI-GRU22_RELU. Acc: 0.8937284850588678


# BIGRU22_RELU_emb200

In [36]:
embedding_size = 200
hidden_size = 64 
num_rnn_layers = 2
num_mlp_layers = 1
model = build_RNN(max_sent_length, len(vocab_dict), embedding_size,
              hidden_size, len(tag_dict),
              num_rnn_layers, num_mlp_layers,
              rnn_type="gru",
              bidirectional=True,
              activation="relu",
              dropout_rate=0.0,
              batch_norm=False,
              l2_reg=0.0,
              loss="categorical_crossentropy",
              optimizer="Adam",
              learning_rate=0.001,
              metric="accuracy")
print(model.summary())

Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 128, 200)          11093800  
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 128, 128)          102144    
_________________________________________________________________
dropout_15 (Dropout)         (None, 128, 128)          0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 128, 128)          74496     
_________________________________________________________________
dropout_16 (Dropout)         (None, 128, 128)          0         
_________________________________________________________________
dense_11 (Dense)             (None, 128, 65)         

In [37]:
#Fine tuning for BI-GRU
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join("models", "weights_BIGRU22_RELU_emb200.hdf5"),
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True)
earlystopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1)

np.random.seed(0)
tf.random.set_seed(0)
rnn_history = model.fit(train_tokens, train_tags,
                    validation_split=0.1,
                    epochs=20, batch_size=100, verbose=1,
                    callbacks=[checkpointer, earlystopping])


Epoch 1/20
213/213 [==============================] - ETA: 0s - loss: 1.0219 - accuracy: 0.7929
Epoch 00001: val_accuracy improved from -inf to 0.85028, saving model to models/weights_BIGRU22_RELU_emb200.hdf5
213/213 [==============================] - 41s 194ms/step - loss: 1.0219 - accuracy: 0.7929 - val_loss: 0.5831 - val_accuracy: 0.8503
Epoch 2/20
213/213 [==============================] - ETA: 0s - loss: 0.4883 - accuracy: 0.8692
Epoch 00002: val_accuracy improved from 0.85028 to 0.87741, saving model to models/weights_BIGRU22_RELU_emb200.hdf5
213/213 [==============================] - 40s 187ms/step - loss: 0.4883 - accuracy: 0.8692 - val_loss: 0.4432 - val_accuracy: 0.8774
Epoch 3/20
213/213 [==============================] - ETA: 0s - loss: 0.3830 - accuracy: 0.8911
Epoch 00003: val_accuracy improved from 0.87741 to 0.88677, saving model to models/weights_BIGRU22_RELU_emb200.hdf5
213/213 [==============================] - 40s 187ms/step - loss: 0.3830 - accuracy: 0.8911 - val_l

In [38]:
model = keras.models.load_model(os.path.join("models", "weights_BIGRU22_RELU_emb200.hdf5"))

train_score = model.evaluate(train_tokens, train_tags,
                             batch_size=100)
test_score = model.evaluate(val_tokens, val_tags,
                            batch_size=100)
print("training loss:", train_score[0], "training accuracy", train_score[1])
print("test loss:", test_score[0], "test accuracy", test_score[1])
train_matrix = model.predict(val_tokens,batch_size=100, verbose=1, callbacks=[checkpointer])
train_tags_by_idx = np.argmax(train_matrix, axis=2)
train_labels = np.array([[idx2tag[p] for p in preds] for preds in train_tags_by_idx])
print(train_labels)
print("BIGRU22_RELU_emb200. Acc:", 
      calc_accuracy(train_labels, 
                    np.array([val_dict['tag_seq']])))

30/30 [==============================] - 1s 33ms/step - loss: 0.3875 - accuracy: 0.9055
training loss: 0.1536434292793274 training accuracy 0.9558627009391785
test loss: 0.38750386238098145 test accuracy 0.9054926037788391
30/30 [==============================] - 1s 22ms/step
[['IMMUNE_RESPONSE' 'O' 'O' ... 'O' 'ORGANISM' 'O']
 ['ORGANISM' 'O' 'O' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ...
 ['O' 'DATE' 'CORONAVIRUS' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ['CHEMICAL' 'O' 'O' ... 'THERAPEUTIC_OR_PREVENTIVE_PROCEDURE' 'O' 'O']]
BIGRU22_RELU_emb200. Acc: 0.8967687812780236


# BIGRU2_RELU_emb200_h128

In [39]:
embedding_size = 200
hidden_size = 128 
num_rnn_layers = 2
num_mlp_layers = 1
model = build_RNN(max_sent_length, len(vocab_dict), embedding_size,
              hidden_size, len(tag_dict),
              num_rnn_layers, num_mlp_layers,
              rnn_type="gru",
              bidirectional=True,
              activation="relu",
              dropout_rate=0.0,
              batch_norm=False,
              l2_reg=0.0,
              loss="categorical_crossentropy",
              optimizer="Adam",
              learning_rate=0.001,
              metric="accuracy")
print(model.summary())

Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 128)]             0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 128, 200)          11093800  
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 128, 256)          253440    
_________________________________________________________________
dropout_17 (Dropout)         (None, 128, 256)          0         
_________________________________________________________________
bidirectional_16 (Bidirectio (None, 128, 256)          296448    
_________________________________________________________________
dropout_18 (Dropout)         (None, 128, 256)          0         
_________________________________________________________________
dense_12 (Dense)             (None, 128, 65)         

In [40]:
#Fine tuning for BI-GRU
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join("models", "weights_BIGRU2_RELU_emb200_h128.hdf5"),
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True)
earlystopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1)

np.random.seed(0)
tf.random.set_seed(0)
rnn_history = model.fit(train_tokens, train_tags,
                    validation_split=0.1,
                    epochs=20, batch_size=100, verbose=1,
                    callbacks=[checkpointer, earlystopping])

Epoch 1/20
213/213 [==============================] - ETA: 0s - loss: 0.9280 - accuracy: 0.8030
Epoch 00001: val_accuracy improved from -inf to 0.85706, saving model to models/weights_BIGRU2_RELU_emb200_h128.hdf5
213/213 [==============================] - 43s 201ms/step - loss: 0.9280 - accuracy: 0.8030 - val_loss: 0.5424 - val_accuracy: 0.8571
Epoch 2/20
213/213 [==============================] - ETA: 0s - loss: 0.4534 - accuracy: 0.8752
Epoch 00002: val_accuracy improved from 0.85706 to 0.88167, saving model to models/weights_BIGRU2_RELU_emb200_h128.hdf5
213/213 [==============================] - 42s 197ms/step - loss: 0.4534 - accuracy: 0.8752 - val_loss: 0.4199 - val_accuracy: 0.8817
Epoch 3/20
213/213 [==============================] - ETA: 0s - loss: 0.3569 - accuracy: 0.8957
Epoch 00003: val_accuracy improved from 0.88167 to 0.89014, saving model to models/weights_BIGRU2_RELU_emb200_h128.hdf5
213/213 [==============================] - 51s 238ms/step - loss: 0.3569 - accuracy: 0.

In [41]:
model = keras.models.load_model(os.path.join("models", "weights_BIGRU2_RELU_emb200_h128.hdf5"))

train_score = model.evaluate(train_tokens, train_tags,
                             batch_size=100)
test_score = model.evaluate(val_tokens, val_tags,
                            batch_size=100)
print("training loss:", train_score[0], "training accuracy", train_score[1])
print("test loss:", test_score[0], "test accuracy", test_score[1])
train_matrix = model.predict(val_tokens,batch_size=100, verbose=1, callbacks=[checkpointer])
train_tags_by_idx = np.argmax(train_matrix, axis=2)
train_labels = np.array([[idx2tag[p] for p in preds] for preds in train_tags_by_idx])
print(train_labels)
print("BIGRU2_RELU_emb200_h128. Acc:", 
      calc_accuracy(train_labels, 
                    np.array([val_dict['tag_seq']])))

30/30 [==============================] - 1s 21ms/step - loss: 0.3760 - accuracy: 0.9075
training loss: 0.15998002886772156 training accuracy 0.9523305296897888
test loss: 0.37604665756225586 test accuracy 0.9075158834457397
30/30 [==============================] - 1s 20ms/step
[['IMMUNE_RESPONSE' 'O' 'O' ... 'O' 'EUKARYOTE' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ['O' 'ORG' 'O' ... 'O' 'O' 'O']
 ...
 ['O' 'DATE' 'CORONAVIRUS' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ['CELL_COMPONENT' 'O' 'O' ... 'THERAPEUTIC_OR_PREVENTIVE_PROCEDURE' 'O'
  'O']]
BIGRU2_RELU_emb200_h128. Acc: 0.8989788538864301


# BIGRU3_RELU_emb200_h128_dr20

In [42]:
embedding_size = 200
hidden_size = 128 
num_rnn_layers = 2
num_mlp_layers = 1
model = build_RNN(max_sent_length, len(vocab_dict), embedding_size,
              hidden_size, len(tag_dict),
              num_rnn_layers, num_mlp_layers,
              rnn_type="gru",
              bidirectional=True,
              activation="relu",
              dropout_rate=0.2,
              batch_norm=False,
              l2_reg=0.0,
              loss="categorical_crossentropy",
              optimizer="Adam",
              learning_rate=0.001,
              metric="accuracy")
print(model.summary())

Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 128)]             0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 128, 200)          11093800  
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 128, 256)          253440    
_________________________________________________________________
dropout_19 (Dropout)         (None, 128, 256)          0         
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 128, 256)          296448    
_________________________________________________________________
dropout_20 (Dropout)         (None, 128, 256)          0         
_________________________________________________________________
dense_13 (Dense)             (None, 128, 65)         

In [43]:
#Fine tuning for BI-GRU
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join("models", "weights_BIGRU3_RELU_emb200_h128_dr20.hdf5"),
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True)
earlystopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1)

np.random.seed(0)
tf.random.set_seed(0)
rnn_history = model.fit(train_tokens, train_tags,
                    validation_split=0.1,
                    epochs=20, batch_size=100, verbose=1,
                    callbacks=[checkpointer, earlystopping])

Epoch 1/20
213/213 [==============================] - ETA: 0s - loss: 0.9418 - accuracy: 0.8004
Epoch 00001: val_accuracy improved from -inf to 0.85604, saving model to models/weights_BIGRU3_RELU_emb200_h128_dr20.hdf5
213/213 [==============================] - 42s 196ms/step - loss: 0.9418 - accuracy: 0.8004 - val_loss: 0.5460 - val_accuracy: 0.8560
Epoch 2/20
213/213 [==============================] - ETA: 0s - loss: 0.4700 - accuracy: 0.8713
Epoch 00002: val_accuracy improved from 0.85604 to 0.88103, saving model to models/weights_BIGRU3_RELU_emb200_h128_dr20.hdf5
213/213 [==============================] - 41s 193ms/step - loss: 0.4700 - accuracy: 0.8713 - val_loss: 0.4239 - val_accuracy: 0.8810
Epoch 3/20
213/213 [==============================] - ETA: 0s - loss: 0.3750 - accuracy: 0.8912
Epoch 00003: val_accuracy improved from 0.88103 to 0.88910, saving model to models/weights_BIGRU3_RELU_emb200_h128_dr20.hdf5
213/213 [==============================] - 40s 188ms/step - loss: 0.3750

In [44]:
model = keras.models.load_model(os.path.join("models", "weights_BIGRU3_RELU_emb200_h128_dr20.hdf5"))

train_score = model.evaluate(train_tokens, train_tags,
                             batch_size=100)
test_score = model.evaluate(val_tokens, val_tags,
                            batch_size=100)
print("training loss:", train_score[0], "training accuracy", train_score[1])
print("test loss:", test_score[0], "test accuracy", test_score[1])
train_matrix = model.predict(val_tokens,batch_size=100, verbose=1, callbacks=[checkpointer])
train_tags_by_idx = np.argmax(train_matrix, axis=2)
train_labels = np.array([[idx2tag[p] for p in preds] for preds in train_tags_by_idx])
print(train_labels)
print("BIGRU3_RELU_emb200_h128_dr20. Acc:", 
      calc_accuracy(train_labels, 
                    np.array([val_dict['tag_seq']])))

30/30 [==============================] - 1s 28ms/step - loss: 0.3768 - accuracy: 0.9053
training loss: 0.16495263576507568 training accuracy 0.9500830769538879
test loss: 0.37676259875297546 test accuracy 0.9053045511245728
30/30 [==============================] - 1s 21ms/step
[['IMMUNE_RESPONSE' 'O' 'O' ... 'O' 'EUKARYOTE' 'O']
 ['ORGANISM' 'ORGANISM' 'O' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ...
 ['O' 'DATE' 'CORONAVIRUS' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ['CELL_COMPONENT' 'O' 'O' ... 'THERAPEUTIC_OR_PREVENTIVE_PROCEDURE' 'O'
  'O']]
BIGRU3_RELU_emb200_h128_dr20. Acc: 0.896563394949232


# BIGRU4_RELU_emb200_h256_dr20

In [46]:
embedding_size = 200
hidden_size = 256
num_rnn_layers = 4
num_mlp_layers = 2
model = build_RNN(max_sent_length, len(vocab_dict), embedding_size,
              hidden_size, len(tag_dict),
              num_rnn_layers, num_mlp_layers,
              rnn_type="gru",
              bidirectional=True,
              activation="relu",
              dropout_rate=0.2,
              batch_norm=False,
              l2_reg=0.0,
              loss="categorical_crossentropy",
              optimizer="Adam",
              learning_rate=0.001,
              metric="accuracy")
print(model.summary())

Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 128)]             0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 128, 200)          11093800  
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 128, 512)          703488    
_________________________________________________________________
dropout_22 (Dropout)         (None, 128, 512)          0         
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 128, 512)          1182720   
_________________________________________________________________
dropout_23 (Dropout)         (None, 128, 512)          0         
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 128, 512)        

In [47]:
#Fine tuning for BI-GRU
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join("models", "weights_BIGRU4_RELU_emb200_h256_dr20.hdf5"),
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True)
earlystopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1)

np.random.seed(0)
tf.random.set_seed(0)
rnn_history = model.fit(train_tokens, train_tags,
                    validation_split=0.1,
                    epochs=20, batch_size=100, verbose=1,
                    callbacks=[checkpointer, earlystopping])

Epoch 1/20
213/213 [==============================] - ETA: 0s - loss: 0.8366 - accuracy: 0.8100
Epoch 00001: val_accuracy improved from -inf to 0.86604, saving model to models/weights_BIGRU4_RELU_emb200_h256_dr20.hdf5
213/213 [==============================] - 67s 315ms/step - loss: 0.8366 - accuracy: 0.8100 - val_loss: 0.4871 - val_accuracy: 0.8660
Epoch 2/20
213/213 [==============================] - ETA: 0s - loss: 0.4270 - accuracy: 0.8791
Epoch 00002: val_accuracy improved from 0.86604 to 0.88651, saving model to models/weights_BIGRU4_RELU_emb200_h256_dr20.hdf5
213/213 [==============================] - 75s 352ms/step - loss: 0.4270 - accuracy: 0.8791 - val_loss: 0.3973 - val_accuracy: 0.8865
Epoch 3/20
213/213 [==============================] - ETA: 0s - loss: 0.3432 - accuracy: 0.8985
Epoch 00003: val_accuracy improved from 0.88651 to 0.89446, saving model to models/weights_BIGRU4_RELU_emb200_h256_dr20.hdf5
213/213 [==============================] - 76s 355ms/step - loss: 0.3432

In [48]:
model = keras.models.load_model(os.path.join("models", "weights_BIGRU4_RELU_emb200_h256_dr20.hdf5"))

train_score = model.evaluate(train_tokens, train_tags,
                             batch_size=100)
test_score = model.evaluate(val_tokens, val_tags,
                            batch_size=100)
print("training loss:", train_score[0], "training accuracy", train_score[1])
print("test loss:", test_score[0], "test accuracy", test_score[1])
train_matrix = model.predict(val_tokens,batch_size=100, verbose=1, callbacks=[checkpointer])
train_tags_by_idx = np.argmax(train_matrix, axis=2)
train_labels = np.array([[idx2tag[p] for p in preds] for preds in train_tags_by_idx])
print(train_labels)
print("BIGRU4_RELU_emb200_h256_dr20. Acc:", 
      calc_accuracy(train_labels, 
                    np.array([val_dict['tag_seq']])))

30/30 [==============================] - 2s 64ms/step - loss: 0.3581 - accuracy: 0.9087
training loss: 0.15418824553489685 training accuracy 0.9533421397209167
test loss: 0.35811764001846313 test accuracy 0.9086679220199585
30/30 [==============================] - 2s 66ms/step
[['IMMUNE_RESPONSE' 'O' 'O' ... 'O' 'ORGANISM' 'O']
 ['ORGANISM' 'ORGANISM' 'O' ... 'O' 'O' 'O']
 ['O' 'O' 'GROUP' ... 'O' 'O' 'O']
 ...
 ['O' 'DATE' 'CORONAVIRUS' ... 'O' 'O' 'O']
 ['O' 'O' 'O' ... 'O' 'O' 'O']
 ['CHEMICAL' 'O' 'O' ... 'O' 'O' 'O']]
BIGRU4_RELU_emb200_h256_dr20. Acc: 0.9002372067459284


# Prediction
Based on the validation accuracy, we have chose BIGRU4_RELU_emb200_h256_dr20 as the final model.

In [49]:
model = keras.models.load_model(os.path.join("models", "weights_BIGRU4_RELU_emb200_h256_dr20.hdf5"))

In [50]:
import json
import pandas as pd

In [53]:
val_matrix = model.predict(val_tokens,batch_size=100, verbose=1, callbacks=[checkpointer])
val_tags_by_idx = np.argmax(val_matrix, axis=2)
val_preds = np.array([[idx2tag[p] for p in preds] for preds in val_tags_by_idx])

30/30 [==============================] - 2s 63ms/step


In [55]:
df = pd.DataFrame({'id': test_dict["id"],
                   'labels': [json.dumps(np.array(preds).tolist()) for preds in val_preds]})
df.to_csv('val_preds.csv', index=False)

In [56]:
pd.read_csv("val_preds.csv")

,id,labels
0,0,"[""IMMUNE_RESPONSE"", ""O"", ""O"", ""GENE_OR_GENOME""..."
1,1,"[""ORGANISM"", ""ORGANISM"", ""O"", ""O"", ""O"", ""O"", ""..."
2,2,"[""O"", ""O"", ""GROUP"", ""O"", ""DISEASE_OR_SYNDROME""..."
3,3,"[""O"", ""O"", ""O"", ""VIRUS"", ""O"", ""O"", ""WILDLIFE"",..."
4,4,"[""EUKARYOTE"", ""VIRUS"", ""O"", ""O"", ""GENE_OR_GENO..."
...,...,...
2945,2945,"[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""GENE..."
2946,2946,"[""O"", ""O"", ""O"", ""IMMUNE_RESPONSE"", ""IMMUNE_RES..."
2947,2947,"[""O"", ""DATE"", ""CORONAVIRUS"", ""O"", ""O"", ""O"", ""O..."
2948,2948,"[""O"", ""O"", ""O"", ""O"", ""CELL_COMPONENT"", ""O"", ""O..."


In [60]:
print("Val accuracy: ", evaluate('val_preds.csv', '/kaggle/input/4901k-project/val.pkl'))

Val accuracy:  0.9002372067459284


Make predictions on test data.

In [61]:
test_matrix = model.predict(test_tokens,batch_size=100, verbose=1, callbacks=[checkpointer])
test_tags_by_idx = np.argmax(test_matrix, axis=2)
test_preds = np.array([[idx2tag[p] for p in preds] for preds in test_tags_by_idx])

30/30 [==============================] - 2s 62ms/step


In [ ]:
test_preds

In [64]:
df = pd.DataFrame({'id': test_dict["id"],
                   'labels': [json.dumps(np.array(preds).tolist()) for preds in test_preds]})
df.to_csv('test_preds.csv', index=False)

In [65]:
pd.read_csv("test_preds.csv")

,id,labels
0,0,"[""O"", ""O"", ""IMMUNE_RESPONSE"", ""IMMUNE_RESPONSE..."
1,1,"[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""_t_pad_"", ""_t_..."
2,2,"[""O"", ""O"", ""O"", ""RESEARCH_ACTIVITY"", ""RESEARCH..."
3,3,"[""O"", ""O"", ""O"", ""CHEMICAL"", ""CHEMICAL"", ""O"", ""..."
4,4,"[""O"", ""CHEMICAL"", ""CHEMICAL"", ""O"", ""O"", ""O"", ""..."
...,...,...
2945,2945,"[""DATE"", ""O"", ""CORONAVIRUS"", ""O"", ""O"", ""O"", ""O..."
2946,2946,"[""CHEMICAL"", ""CHEMICAL"", ""CHEMICAL"", ""CHEMICAL..."
2947,2947,"[""O"", ""O"", ""DISEASE_OR_SYNDROME"", ""CHEMICAL"", ..."
2948,2948,"[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ..."
